In [1]:
import sys, yfinance, yaml, pandas, pyarrow
print("Импорты прошли! Работаю из", sys.executable)



Импорты прошли! Работаю из /opt/anaconda3/envs/stocks/bin/python


In [3]:
import pathlib
pathlib.Path("data/raw").mkdir(parents=True, exist_ok=True)
print("Папка data/raw готова!")


Папка data/raw готова!


In [5]:
import pandas as pd, yaml, requests

url = "https://gist.githubusercontent.com/ZeccaLehn/f6a2613b24c393821f81c0c1d23d4192/raw/SP500.csv"
sp500 = pd.read_csv(url)
tickers = sorted(sp500["Symbol"].unique())

with open("universe.yaml", "w") as f:
    yaml.safe_dump({"tickers": tickers}, f)

print(f"{len(tickers)} tickers saved to universe.yaml")


505 tickers saved to universe.yaml


In [7]:
import yfinance as yf, yaml, pandas as pd, pathlib

pathlib.Path("data/raw").mkdir(parents=True, exist_ok=True)

tickers = yaml.safe_load(open("universe.yaml"))["tickers"]
prices = yf.download(tickers,
                     start="2015-01-01",
                     auto_adjust=True,
                     progress=False)["Close"]

prices.to_parquet("data/raw/prices.parquet")
print("Saved", prices.shape, "to data/raw/prices.parquet")



75 Failed downloads:
['TMK', 'CTXS', 'HRS', 'CERN', 'ATVI', 'MRO', 'CTL', 'RE', 'MON', 'CXO', 'CHK', 'ADS', 'ABC', 'DRE', 'FISV', 'PXD', 'BRK.B', 'RHT', 'KSU', 'NBL', 'PKI', 'MYL', 'DWDP', 'JEC', 'TIF', 'WRK', 'XEC', 'FB', 'DISCA', 'VIAB', 'PBCT', 'XL', 'BHGE', 'ALXN', 'ARNC', 'TSS', 'DISH', 'RTN', 'FBHS', 'BBT', 'DISCK', 'COG', 'BLL', 'SYMC', 'WLTW', 'NLSN', 'ANTM', 'HCP', 'XLNX', 'AGN', 'VAR', 'ETFC', 'CBS', 'UTX', 'LLL', 'CELG', 'GPS', 'FLIR', 'APC']: YFTzMissingError('possibly delisted; no timezone found')
['HCN', 'BF.B', 'DPS', 'WYN', 'PCLN', 'CBG', 'LUK', 'GGP', 'SNI', 'KORS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-05-21)')
['TWX', 'CSRA', 'ANDV', 'AET', 'SCG']: ValueError('Length mismatch: Expected axis has 2 elements, new values have 1 elements')
['CRM']: Timeout('Failed to perform, curl: (28) Operation timed out after 10002 milliseconds with 65026 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more de

Saved (2612, 505) to data/raw/prices.parquet


In [14]:
import pathlib, pandas as pd

prices = pd.read_parquet("data/raw/prices.parquet")
print("Before drop:", prices.shape)

all_nan = prices.isna().all()
prices = prices.loc[:, ~all_nan]

print("After drop :", prices.shape)

# ---- вот эта строка создаст папку, если её нет ----
pathlib.Path("data/processed").mkdir(parents=True, exist_ok=True)

prices.to_parquet("data/processed/prices_raw_clean.parquet")
print("Файл сохранён!")


Before drop: (2612, 505)
After drop : (2612, 430)
Файл сохранён!


In [16]:
prices = pd.read_parquet("data/raw/prices.parquet")
print("Before drop:", prices.shape)

all_nan = prices.isna().all()
prices = prices.loc[:, ~all_nan]

print("After drop :", prices.shape)
prices.to_parquet("data/processed/prices_raw_clean.parquet")


Before drop: (2612, 505)
After drop : (2612, 430)


In [19]:
prices = prices.sort_index()          # вдруг даты перемешались
prices = prices.ffill().bfill()       # вперёд/назад заполнение

# sanity-check
assert prices.isna().sum().sum() == 0, "Остались пропуски!"

prices.to_parquet("data/processed/prices_filled.parquet")


In [21]:
returns = prices.pct_change().dropna(how="all")
returns.to_parquet("data/processed/returns.parquet")
print("Returns shape:", returns.shape)


Returns shape: (2611, 430)


In [23]:
print("Mean daily return:", returns.mean().mean().round(4))
print("Mean daily volatility:", returns.std().mean().round(4))


Mean daily return: 0.0005
Mean daily volatility: 0.0201


In [6]:
import openbb                # ← корневой пакет
from openbb import obb       # ← объект-“приложение”

print("OpenBB version:", openbb.__version__)   # ✓ так работает
# или просто пропустите проверку версии

# запишем ключ FMP
obb.user.set_apikey("fmp", "zqeoRJ2nkCUIYhe6q6tRLSTzePdObTuC")   # токен лучше сразу сменить
obb.user.save_settings()                       # сохранит в ~/.openbb

print("Ключ сохранён:", obb.user.get_apikey("fmp")[:4]+"…")


AttributeError: module 'openbb' has no attribute '__version__'

In [8]:
# --- импортируем ---
from openbb import obb                    # главный объект-«приложение»
import importlib.metadata as md           # стандартный модуль Python ≥3.8

# безопасно узнаём версию пакета (если очень хочется)
try:
    ver = md.version("openbb")
    print("OpenBB version:", ver)
except md.PackageNotFoundError:
    print("OpenBB: версия не объявлена, но пакет загружен")

# --- прописываем FMP-ключ ---
obb.user.set_apikey("fmp", "zqeoRJ2nkCUIYhe6q6tRLSTzePdObTuC")   # лучше сразу регенерировать
obb.user.save_settings()
print("Ключ сохранён локально!")

# --- быстрая проверка: профиль Apple ---
df = obb.equity.profile("AAPL", provider="fmp")
display(df.head())


OpenBB version: 4.4.4


AttributeError: 'UserSettings' object has no attribute 'set_apikey'

In [10]:
import os
# ❗️ вставьте НОВЫЙ сгенерированный токен – тот, что был на скрине, лучше уже заменить
os.environ["FMP_API_KEY"] = "zqeoRJ2nkCUIYhe6q6tRLSTzePdObTuC"


In [4]:
import pathlib, os, textwrap, sys

cred_path = pathlib.Path.home() / ".openbb" / "credentials.toml"
print("Ожидаемый путь:", cred_path)
print("Файл существует? ➜", cred_path.exists())

if cred_path.exists():
    print("\nСодержимое файла:")
    print(textwrap.indent(cred_path.read_text(), "    "))


Ожидаемый путь: /Users/malekamerzhoeva/.openbb/credentials.toml
Файл существует? ➜ True

Содержимое файла:
    fmp_api_key = "zqeoRJ2nkCUIYhe6q6tRLSTzePdObTuC"

    A



In [6]:
fmp_api_key = "zqeoRJ2nkCUIYhe6q6tRLSTzePdObTuC"


In [2]:
from openbb import obb
obb.equity.profile("AAPL", provider="fmp").head()


OpenBBError: 
[Error] -> Missing credential 'fmp_api_key'. Check https://financialmodelingprep.com to get it. Known more about how to set provider credentials at https://docs.openbb.co/platform/getting_started/api_keys.